<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/Copy_of_model_load_work_transformer_demo_fp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Model environmenmt

In [ ]:
!pip install colab-env -q

In [4]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [ ]:
!pip install transformers datasets torch geopy -q

In [ ]:
!pip show transformers
print()
!pip show datasets
print()
!pip show torch

## training

In [7]:
import random
import math
from datasets import Dataset
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from tqdm import tqdm
from geopy.point import Point  # Import Point object
import warnings
warnings.filterwarnings("ignore")
import torch
import re

# Define airport codes and names
airports = [
    # North America
    "ATL", "LAX", "ORD", "DFW", "DEN", "JFK", "SFO", "LAS", "SEA", "CLT",
    "MIA", "PHL", "EWR", "BOS", "PHX", "MCO", "IAH", "DAL", "FLL", "DTW",
    "MSP", "IAD", "BWI", "SLC", "SAN", "MDW", "TPA", "PDX", "BNA", "STL",
    # Europe
    "LHR", "CDG", "AMS", "FRA", "MAD", "BCN", "FCO", "MUC", "DUB", "ZRH",
    # Asia
    "HND", "PVG", "ICN", "SIN", "KUL", "BKK", "DXB", "DEL", "HKG", "NRT",
    # Caribbean
    "CUN", "PUJ", "SJU", "MBJ", "NAS",
    # South America
    "GRU", "BOG", "EZE", "SCL", "LIM",
    # Australia & New Zealand
    "SYD", "MEL", "BNE", "PER", "AKL"
]

airport_names = {
    "ATL": "Hartsfield-Jackson Atlanta International Airport",
    "LAX": "Los Angeles International Airport",
    "ORD": "Chicago O'Hare International Airport",
    "DFW": "Dallas/Fort Worth International Airport",
    "DEN": "Denver International Airport",
    "JFK": "John F. Kennedy International Airport",
    "SFO": "San Francisco International Airport",
    "LAS": "Harry Reid International Airport",
    "SEA": "Seattle-Tacoma International Airport",
    "CLT": "Charlotte Douglas International Airport",
    "MIA": "Miami International Airport",
    "PHL": "Philadelphia International Airport",
    "EWR": "Newark Liberty International Airport",
    "BOS": "Logan International Airport",
    "PHX": "Phoenix Sky Harbor International Airport",
    "MCO": "Orlando International Airport",
    "IAH": "George Bush Intercontinental Airport",
    "DAL": "Dallas Love Field",
    "FLL": "Fort Lauderdale-Hollywood International Airport",
    "DTW": "Detroit Metropolitan Airport",
    "MSP": "Minneapolis-Saint Paul International Airport",
    "IAD": "Washington Dulles International Airport",
    "BWI": "Baltimore/Washington International Airport",
    "SLC": "Salt Lake City International Airport",
    "SAN": "San Diego International Airport",
    "MDW": "Midway International Airport",
    "TPA": "Tampa International Airport",
    "PDX": "Portland International Airport",
    "BNA": "Nashville International Airport",
    "STL": "St. Louis Lambert International Airport",
    # Europe
    "LHR": "London Heathrow Airport",
    "CDG": "Charles de Gaulle Airport",
    "AMS": "Amsterdam Airport Schiphol",
    "FRA": "Frankfurt Airport",
    "MAD": "Adolfo Suárez Madrid–Barajas Airport",
    "BCN": "Barcelona–El Prat Airport",
    "FCO": "Leonardo da Vinci–Fiumicino Airport",
    "MUC": "Munich Airport",
    "DUB": "Dublin Airport",
    "ZRH": "Zurich Airport",
    # Asia
    "HND": "Haneda Airport",
    "PVG": "Shanghai Pudong International Airport",
    "ICN": "Incheon International Airport",
    "SIN": "Singapore Changi Airport",
    "KUL": "Kuala Lumpur International Airport",
    "BKK": "Suvarnabhumi Airport",
    "DXB": "Dubai International Airport",
    "DEL": "Indira Gandhi International Airport",
    "HKG": "Hong Kong International Airport",
    "NRT": "Narita International Airport",
    # Caribbean
    "CUN": "Cancún International Airport",
    "PUJ": "Punta Cana International Airport",
    "SJU": "Luis Muñoz Marín International Airport",
    "MBJ": "Sangster International Airport",
    "NAS": "Lynden Pindling International Airport",
    # South America
    "GRU": "São Paulo–Guarulhos International Airport",
    "BOG": "El Dorado International Airport",
    "EZE": "Ministro Pistarini International Airport",
    "SCL": "Arturo Merino Benítez International Airport",
    "LIM": "Jorge Chávez International Airport",
    # Australia & New Zealand
    "SYD": "Sydney Airport",
    "MEL": "Melbourne Airport",
    "BNE": "Brisbane Airport",
    "PER": "Perth Airport",
    "AKL": "Auckland Airport"
}

In [ ]:
from transformers import AutoTokenizer
tokenizer_name = 'gpt2'
#tokenizer_name = 'meta-llama/Meta-Llama-3-8B'
#tokenizer_name = 'mistralai/Mistral-7B-Instruct-v0.3'
pad_token = '<pad>'

# --- Load Tokenizer and Add Pad Token ---
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': pad_token})
# Ensure pad_token_id is within the vocabulary size
pad_token_id = tokenizer.pad_token_id
if pad_token_id >= tokenizer.vocab_size:
    pad_token_id = tokenizer.vocab_size - 1

In [ ]:
from datasets import load_dataset
flight_plan_dataset = load_dataset("frankmorales2020/flight_plan_waypoints")
flight_plan_dataset['train'][7]

In [ ]:
!pip install transformers datasets torch geopy -q

from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

# Configuration
tokenizer_name = "gpt2"
batch_size = 4
sequence_length = 128  # Adjusted based on tokenize_function
embedding_dimension = 256
num_heads = 8
feed_forward_dimension = 1024
num_encoder_layers = 6
dropout_probability = 0.3
learning_rate = 2e-5
num_epochs = 10

# Load Tokenizer and Add Pad Token (if needed)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<pad>'})

# Ensure pad_token_id is within the vocabulary size
pad_token_id = tokenizer.pad_token_id
if pad_token_id >= tokenizer.vocab_size:
    pad_token_id = tokenizer.vocab_size - 1  # Clip to maximum vocabulary index


# 1. Tokenize and Format
def tokenize_function(examples):
    # Directly use waypoint numbers as labels
    examples["labels"] = examples["label"]
    tokenized_output = tokenizer(
        examples["input"],
        padding="max_length",
        truncation=True,
        max_length=sequence_length
    )

    # Instead of assigning labels directly, shift them for causal LM
    tokenized_output['labels'] = tokenized_output['input_ids'].copy()

    # Replace input_ids corresponding to pad_token with -100 in labels
    tokenized_output['labels'] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label_list]
        for label_list in tokenized_output['labels']
    ]

    # --- Ensure input_ids are within vocabulary range before padding ---
    tokenized_output['input_ids'] = [
        [token if token < tokenizer.vocab_size else tokenizer.unk_token_id for token in label_list]
        for label_list in tokenized_output['input_ids']
    ]

    # --- Additional Checks ---
    # Ensure that labels are within the vocabulary range as well
    tokenized_output['labels'] = [
        [token if token < tokenizer.vocab_size else tokenizer.unk_token_id for token in label_list]
        for label_list in tokenized_output['labels']
    ]

    # Handle case where labels are empty (could happen due to errors in data)
    for i, label_list in enumerate(tokenized_output['labels']):
        if len(label_list) == 0:
            # Replace with a single UNK token or handle as appropriate
            tokenized_output['labels'][i] = [tokenizer.unk_token_id]

    return tokenized_output


# Load dataset
dataset = load_dataset("frankmorales2020/flight_plan_waypoints")

# Tokenize, format, and split dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["input", "label"])
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# --- Split the 'train' dataset within the DatasetDict ---
train_testvalid = tokenized_datasets['train'].train_test_split(test_size=0.2, seed=42)

# Create new DatasetDict with the splits
tokenized_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'test': train_testvalid['test']
})

# Further split the 'test' set into validation and test
test_valid = tokenized_datasets['test'].train_test_split(test_size=0.5, seed=42)

# Update the DatasetDict
tokenized_datasets = DatasetDict({
    'train': tokenized_datasets['train'],
    'validation': test_valid['test'], # Renamed to 'validation'
    'test': test_valid['train'] # Renamed to 'test'
})


train_dataset = tokenized_datasets["train"]
testvalid_dataset = tokenized_datasets["test"]  # Assuming you still need this
eval_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

small_train_dataset = train_dataset.shuffle(seed=42).select(range(1600))
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(200))

# 2. Data Loading
train_dataloader = DataLoader(small_train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=batch_size)

# 3. Model
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_hidden_dim, num_layers, block_size, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(block_size, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(embed_dim, num_heads, ff_hidden_dim, dropout)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.linear_out = nn.Linear(embed_dim, vocab_size)  # Output layer for token prediction
        self.block_size = block_size
        self.dropout = nn.Dropout(dropout)

        # --- Initialize weights to handle potential out-of-vocabulary tokens ---
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data) * 0.02  # Example initialization

    def forward(self, input_ids, attention_mask):
        # 1. Ensure input_ids are within vocabulary range
        input_ids = input_ids.clamp(0, tokenizer.vocab_size - 1)

        # 2. Create positional embeddings - Adjusted to use batch size
        batch_size = input_ids.shape[0]
        positions = torch.arange(0, input_ids.size(1), dtype=torch.long, device=input_ids.device)
        positions = positions.unsqueeze(0).repeat(batch_size, 1) # Repeat for each item in batch
        pos_embeddings = self.pos_embedding(positions)

        # 3. Combine embeddings
        embeddings = self.dropout(self.embedding(input_ids) + pos_embeddings)

        # 4. Create attention mask
        # attention_mask = input_ids != tokenizer.pad_token_id  # Not needed

        # 5. Ensure attention_mask is boolean and has correct shape
        #attention_mask = attention_mask.type(torch.bool)

        # Create src_key_padding_mask with correct shape
        src_key_padding_mask = (input_ids == tokenizer.pad_token_id)

        # 6. Pass to encoder
        encoder_output = self.encoder(embeddings, src_key_padding_mask=src_key_padding_mask)

        # 7. Output layer
        output = self.linear_out(encoder_output)

        return output

model = SimpleTransformer(
    vocab_size=tokenizer.vocab_size,
    embed_dim=embedding_dimension,
    num_heads=num_heads,
    ff_hidden_dim=feed_forward_dimension,
    num_layers=num_encoder_layers,
    block_size=sequence_length,
    dropout=dropout_probability
)

# 4. Loss and Optimizer
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 5. Training and Evaluation Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs.view(-1, tokenizer.vocab_size), labels.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Average Training Loss: {avg_train_loss:.4f}")

    # Evaluation Loop
    model.eval()
    total_eval_loss = 0

    with torch.no_grad():
        for batch in eval_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs.view(-1, tokenizer.vocab_size), labels.view(-1))
            total_eval_loss += loss.item()

    avg_eval_loss = total_eval_loss / len(eval_dataloader)
    print(f"Epoch {epoch + 1} - Average Evaluation Loss: {avg_eval_loss:.4f}")

## Model Deployment To HF

In [ ]:
!pip install --upgrade huggingface_hub -q
from huggingface_hub import notebook_login, create_repo, upload_file

In [ ]:
from huggingface_hub import notebook_login, create_repo, upload_file

repo_name = "frankmorales2020/FlightPlan_Transformer_LLM"

# --- Automatically generate README.md ---
readme_content = f"""
# Flight Plan Transformer

This repository contains a Transformer model trained to generate flight plan waypoints based on textual input. The model was trained using the "frankmorales2020/flight_plan_waypoints" dataset from Hugging Face Datasets.

## Model Description

The model is a decoder-only Transformer architecture implemented using PyTorch and the `transformers` library. It takes textual input describing the flight (e.g., "Calculate the waypoints from SIN to CUN. Departure: 2024-06-19, Aircraft: Airbus A320, Weather: Partly Cloudy") and predicts a sequence of waypoints representing the flight path.

## Intended Use

This model can be used to:

- **Generate flight plans:** Given flight details, the model can predict waypoints for the route.
- **Flight planning assistance:** It can serve as a tool to aid in flight planning, providing suggestions for waypoints.

## Limitations

- **Accuracy:** The model's accuracy is limited by the quality and coverage of the training data. It may not be perfectly accurate in all scenarios.
- **Real-world applicability:** This model is a research prototype and should not be used for actual flight navigation without thorough validation and safety checks.

## How to Use

1. **Install the necessary libraries:**
Use code with caution
bash pip install transformers datasets torch

2. **Load the model and tokenizer:**
Use code with caution
python from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("{repo_name}") model = AutoModelForSeq2SeqLM.from_pretrained("{repo_name}")

3. **Generate waypoints:**
Use code with caution
python def generate_flight_plan(model, tokenizer, start_text, max_new_tokens=200):

# ... (Your generation code from the original script) ...
4. **Example usage:**
Use code with caution
python query = "Calculate the waypoints from SIN to CUN. Departure: 2024-06-19, Aircraft: Airbus A320, Weather: Partly Cloudy" generated_plan = generate_flight_plan(model, tokenizer, start_text=query, max_new_tokens=256) print(generated_plan)

## Training Data

The model was trained on the "frankmorales2020/flight_plan_waypoints" dataset from Hugging Face Datasets. This dataset contains flight plans with corresponding waypoints.

## Evaluation

The model's performance was evaluated on a held-out portion of the training data. (Include specific metrics or evaluation results if available)

## Acknowledgements

- Thanks to Hugging Face for providing the `transformers` and `datasets` libraries.
- Thanks to frankmorales2020 for creating and sharing the flight plan dataset.

## Contact
Frank Morales, BEng, MEng, SMIEEE
Boeing Associate Technical Fellow
Linkedin: https://www.linkedin.com/in/frank-morales1964/
"""

# Write README.md to file
with open("README.md", "w") as f:
    f.write(readme_content)

In [ ]:
import os
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")
token=access_token_write

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.get_token_permission(token=access_token_write)
repo_id = 'frankmorales2020/FlightPlan_Transformer_LLM'
api.delete_repo(repo_id=repo_id)

In [ ]:
from huggingface_hub import HfApi, HfFolder
import os
import shutil

# --- Push to Hugging Face Hub ---

repo_name = "frankmorales2020/FlightPlan_Transformer_LLM"
create_repo(repo_name, private=False, exist_ok=True)

# --- Push the model directly from memory ---
api = HfApi()
api.upload_file(
    path_or_fileobj="README.md",
    path_in_repo="README.md",
    repo_id=repo_name,
    repo_type="model",
)

# Save the model and tokenizer to a temporary directory
model_save_dir = "flight_plan_transformer_model"
os.makedirs(model_save_dir, exist_ok=True)

# 1. Create a configuration dictionary for your model
config = {
    "vocab_size": tokenizer.vocab_size,
    "embed_dim": embedding_dimension,  # Use your embedding dimension variable
    "num_heads": num_heads,  # Use your num_heads variable
    "ff_hidden_dim": feed_forward_dimension,  # Use your feed_forward_dimension variable
    "num_layers": num_encoder_layers,  # Use your num_encoder_layers variable
    "block_size": block_size,  # Use your block_size variable
    "dropout": dropout_probability  # Use your dropout_probability variable
}

# 2. Save the configuration to config.json
with open(os.path.join(model_save_dir, "config.json"), "w") as f:
    json.dump(config, f)

# Save the model's state_dict instead of using save_pretrained
torch.save(model.state_dict(), os.path.join(model_save_dir, 'pytorch_model.bin'))
tokenizer.save_pretrained(model_save_dir) # This is fine as tokenizer is from Hugging Face


# Upload the model files, including config.json and pytorch_model.bin
upload_file(
    path_or_fileobj=os.path.join(model_save_dir, "config.json"),
    path_in_repo="config.json",
    repo_id=repo_name,
)
upload_file(
    path_or_fileobj=os.path.join(model_save_dir, "pytorch_model.bin"),
    path_in_repo="pytorch_model.bin",
    repo_id=repo_name,
)

# Upload other files (e.g., tokenizer files)
for filename in os.listdir(model_save_dir):
    if filename not in ["config.json", "pytorch_model.bin"]:  # Exclude already uploaded files
        upload_file(
            path_or_fileobj=os.path.join(model_save_dir, filename),
            path_in_repo=filename,
            repo_id=repo_name,
        )


# Remove the temporary directory (optional)
shutil.rmtree(model_save_dir)

print(f"Model pushed to Hugging Face Hub: {repo_name}")

## Model Evaluation

In [ ]:
from transformers import AutoTokenizer, AutoConfig
import torch
import json
from huggingface_hub import hf_hub_download

# --- Updated Model Loading ---
pretrained_model_name = "frankmorales2020/FlightPlan_Transformer_LLM"

# 1. Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

# 2. Load configuration using hf_hub_download
config_path = hf_hub_download(repo_id=pretrained_model_name, filename="config.json")
with open(config_path, 'r') as f:
    config_dict = json.load(f)

# *** Add max_waypoints to the config dictionary or load it from elsewhere ***
config_dict['max_waypoints'] = 10  # Replace with your actual max_waypoints value

# 3. Instantiate your SimpleTransformer class using the loaded configuration
model = SimpleTransformer(
    vocab_size=config_dict['vocab_size'],
    embed_dim=config_dict['embed_dim'],
    num_heads=config_dict['num_heads'],
    ff_hidden_dim=config_dict['ff_hidden_dim'],
    num_layers=config_dict['num_layers'],
    block_size=config_dict['block_size'],
    dropout=config_dict['dropout'],
    max_waypoints=config_dict['max_waypoints'] # Pass max_waypoints here
)

# 4. Load the model weights using hf_hub_download
model_path = hf_hub_download(repo_id=pretrained_model_name, filename="pytorch_model.bin")
state_dict = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

In [ ]:
print(model)

In [ ]:
# --- Load and Preprocess Flight Plan Data ---
flight_plan_dataset = load_dataset("frankmorales2020/flight_plan_waypoints")

# Split into train and validation sets
flight_plan_dataset = flight_plan_dataset['train'].train_test_split(test_size=0.2, seed=42)
flight_plan_dataset = DatasetDict({
    'train': flight_plan_dataset['train'],
    'validation': flight_plan_dataset['test']
})

In [ ]:
flight_plan_dataset['train'][1]

In [ ]:
import random
import math
from datasets import Dataset
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from tqdm import tqdm
from geopy.point import Point  # Import Point object
import warnings
warnings.filterwarnings("ignore")
import torch
import re

# Define airport codes and names
airports = [
    # North America
    "ATL", "LAX", "ORD", "DFW", "DEN", "JFK", "SFO", "LAS", "SEA", "CLT",
    "MIA", "PHL", "EWR", "BOS", "PHX", "MCO", "IAH", "DAL", "FLL", "DTW",
    "MSP", "IAD", "BWI", "SLC", "SAN", "MDW", "TPA", "PDX", "BNA", "STL",
    # Europe
    "LHR", "CDG", "AMS", "FRA", "MAD", "BCN", "FCO", "MUC", "DUB", "ZRH",
    # Asia
    "HND", "PVG", "ICN", "SIN", "KUL", "BKK", "DXB", "DEL", "HKG", "NRT",
    # Caribbean
    "CUN", "PUJ", "SJU", "MBJ", "NAS",
    # South America
    "GRU", "BOG", "EZE", "SCL", "LIM",
    # Australia & New Zealand
    "SYD", "MEL", "BNE", "PER", "AKL"
]

airport_names = {
    "ATL": "Hartsfield-Jackson Atlanta International Airport",
    "LAX": "Los Angeles International Airport",
    "ORD": "Chicago O'Hare International Airport",
    "DFW": "Dallas/Fort Worth International Airport",
    "DEN": "Denver International Airport",
    "JFK": "John F. Kennedy International Airport",
    "SFO": "San Francisco International Airport",
    "LAS": "Harry Reid International Airport",
    "SEA": "Seattle-Tacoma International Airport",
    "CLT": "Charlotte Douglas International Airport",
    "MIA": "Miami International Airport",
    "PHL": "Philadelphia International Airport",
    "EWR": "Newark Liberty International Airport",
    "BOS": "Logan International Airport",
    "PHX": "Phoenix Sky Harbor International Airport",
    "MCO": "Orlando International Airport",
    "IAH": "George Bush Intercontinental Airport",
    "DAL": "Dallas Love Field",
    "FLL": "Fort Lauderdale-Hollywood International Airport",
    "DTW": "Detroit Metropolitan Airport",
    "MSP": "Minneapolis-Saint Paul International Airport",
    "IAD": "Washington Dulles International Airport",
    "BWI": "Baltimore/Washington International Airport",
    "SLC": "Salt Lake City International Airport",
    "SAN": "San Diego International Airport",
    "MDW": "Midway International Airport",
    "TPA": "Tampa International Airport",
    "PDX": "Portland International Airport",
    "BNA": "Nashville International Airport",
    "STL": "St. Louis Lambert International Airport",
    # Europe
    "LHR": "London Heathrow Airport",
    "CDG": "Charles de Gaulle Airport",
    "AMS": "Amsterdam Airport Schiphol",
    "FRA": "Frankfurt Airport",
    "MAD": "Adolfo Suárez Madrid–Barajas Airport",
    "BCN": "Barcelona–El Prat Airport",
    "FCO": "Leonardo da Vinci–Fiumicino Airport",
    "MUC": "Munich Airport",
    "DUB": "Dublin Airport",
    "ZRH": "Zurich Airport",
    # Asia
    "HND": "Haneda Airport",
    "PVG": "Shanghai Pudong International Airport",
    "ICN": "Incheon International Airport",
    "SIN": "Singapore Changi Airport",
    "KUL": "Kuala Lumpur International Airport",
    "BKK": "Suvarnabhumi Airport",
    "DXB": "Dubai International Airport",
    "DEL": "Indira Gandhi International Airport",
    "HKG": "Hong Kong International Airport",
    "NRT": "Narita International Airport",
    # Caribbean
    "CUN": "Cancún International Airport",
    "PUJ": "Punta Cana International Airport",
    "SJU": "Luis Muñoz Marín International Airport",
    "MBJ": "Sangster International Airport",
    "NAS": "Lynden Pindling International Airport",
    # South America
    "GRU": "São Paulo–Guarulhos International Airport",
    "BOG": "El Dorado International Airport",
    "EZE": "Ministro Pistarini International Airport",
    "SCL": "Arturo Merino Benítez International Airport",
    "LIM": "Jorge Chávez International Airport",
    # Australia & New Zealand
    "SYD": "Sydney Airport",
    "MEL": "Melbourne Airport",
    "BNE": "Brisbane Airport",
    "PER": "Perth Airport",
    "AKL": "Auckland Airport"
}

In [ ]:
# ... (other imports and definitions) ...

def generate_flight_plan(model, tokenizer, start_text, max_new_tokens=256, device='cuda'):
    model.eval()  # Set model to evaluation mode
    model.to(device)  # Ensure the model is on the correct device

    # Tokenize input
    start_tokens = tokenizer(start_text, return_tensors='pt', padding=True, truncation=True)

    # 1. Clip input_ids
    start_tokens['input_ids'] = start_tokens['input_ids'].clip(0, tokenizer.vocab_size - 1).type(torch.int64)

    # 2. Create a new attention_mask with correct shape and type
    seq_len = start_tokens['input_ids'].shape[1]
    start_tokens['attention_mask'] = torch.ones(1, seq_len, dtype=torch.bool, device=device) # Directly on device

    # 3. Move other tensors to device
    start_tokens['input_ids'] = start_tokens['input_ids'].to(device)

    # Generate waypoints
    with torch.no_grad():
        try:
            # *** Get the number of waypoints from input or another source ***
            num_waypoints = torch.tensor([5], dtype=torch.long, device=device)  # Assuming 5 waypoints here

            # Now call model with num_waypoints as a Tensor
            predicted_waypoints = model(start_tokens['input_ids'], start_tokens['attention_mask'], num_waypoints)

            # Check if model generated empty or invalid waypoints
            if predicted_waypoints.shape[0] == 0 or predicted_waypoints.shape[1] == 0 or torch.isnan(predicted_waypoints).any():
                print("Warning: Model generated empty or invalid waypoints.")
                return []  # Return an empty list if waypoints are empty or invalid

        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"Error: {e}. Try reducing batch size or sequence length.")
                torch.cuda.empty_cache()  # Clear CUDA memory
                return []  # Return an empty list to avoid hanging
            else:
                raise  # Re-raise the error if it's not memory-related

    # Process waypoints and return as a list
    waypoints_list = predicted_waypoints.squeeze(0).cpu().tolist()  # Move to CPU before converting to list

    # --- Debugging Prints ---
    print("\n--- generate_flight_plan ---")
    print("start_text:", start_text)
    print("start_tokens['input_ids'] shape:", start_tokens['input_ids'].shape)
    print("start_tokens['attention_mask'] shape:", start_tokens['attention_mask'].shape)
    print("num_waypoints:", num_waypoints)
    print("predicted_waypoints shape:", predicted_waypoints.shape)
    print("waypoints_list:", waypoints_list)

    return waypoints_list

# ... (rest of the code - same as before) ...

In [ ]:
#query='Calculate the waypoints from DXB to MSP. Departure: 2024-05-03, Aircraft: Boeing 777, Weather: Overcast'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
query= 'Calculate the waypoints from LHR to AMS. Departure: 2024-06-15, Aircraft: Piper PA-28, Weather: Partly Cloudy'
print(f"Query: {query}")
generated_plan = generate_flight_plan(model, tokenizer, start_text=query, device=device) # Pass the dataset here
print("\nGenerated waypoint list:")
print(generated_plan)